In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
from keras.preprocessing import image
pd.options.display.max_columns = 50

In [27]:
train_df = pd.read_csv('/kaggle/input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv')
val_df = pd.read_csv('/kaggle/input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv')
test_df = pd.read_csv('/kaggle/input/retinal-disease-classification/Test_Set/Test_Set/RFMiD_Testing_Labels.csv')

In [5]:
train_df['img_path'] = [f'/kaggle/input/retinal-disease-classification/Training_Set/Training_Set/Training/{id}.png' for id in train_df['ID']]
val_df['img_path'] = [f'/kaggle/input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/{id}.png' for id in val_df['ID']]
test_df['img_path'] = [f'/kaggle/input/retinal-disease-classification/Test_Set/Test_Set/Test/{id}.png' for id in test_df['ID']]

In [9]:
train_df = train_df.drop(labels=['ID'],axis=1)

In [11]:
train_df = train_df.drop(labels=['Disease_Risk','ODPM', 'HR'], axis=1)
val_df = val_df.drop(labels=['ID', 'Disease_Risk', 'ODPM', 'HR'], axis=1)
test_df = test_df.drop(labels=['ID', 'Disease_Risk', 'ODPM', 'HR'], axis=1)

In [12]:
train_df.columns

Index(['DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN', 'ERM', 'LS', 'MS',
       'CSR', 'ODC', 'CRVO', 'TV', 'AH', 'ODP', 'ODE', 'ST', 'AION', 'PT',
       'RT', 'RS', 'CRS', 'EDN', 'RPEC', 'MHL', 'RP', 'CWS', 'CB', 'PRH',
       'MNF', 'CRAO', 'TD', 'CME', 'PTCR', 'CF', 'VH', 'MCA', 'VS', 'BRAO',
       'PLQ', 'HPED', 'CL', 'img_path'],
      dtype='object')

In [14]:
def weight_calc(col):
    total = len(train_df)
    weight = (1 / train_df[col].sum()) *  total / 2
    return weight

In [15]:
Y_train = list(train_df.drop(['img_path'], axis=1).columns)
Y_val = list(val_df.drop(['img_path'], axis=1).columns)
Y_test = list(test_df.drop(['img_path'], axis=1).columns)
unq_disease = len(Y_train)

In [16]:
class_weights = {}
for i in range(0, unq_disease):
    class_weights[f'{i}'] = weight_calc(Y_train[i])

In [17]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                horizontal_flip=True,
                                                                vertical_flip=True,
                                                                rotation_range=90,
                                                                brightness_range=[0, 0.1])
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [18]:
train_gen = train_datagen.flow_from_dataframe(train_df, 
                                              x_col='img_path', 
                                              y_col=Y_train,
                                              target_size=(150,150),
                                              class_mode='raw',
                                              batch_size=16,
                                              shuffle=True)

Found 1920 validated image filenames.


In [19]:
val_gen = val_datagen.flow_from_dataframe(val_df,
                                          x_col='img_path',
                                          y_col=Y_val,
                                          target_size=(150,150),
                                          class_mode='raw',
                                          batch_size=8)

Found 640 validated image filenames.


In [20]:
test_gen = test_datagen.flow_from_dataframe(test_df,
                                            x_col='img_path',
                                            y_col=Y_test,
                                            target_size=(150,150),
                                            class_mode='raw')

Found 640 validated image filenames.


In [21]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [22]:
pre_trained_model = InceptionV3(input_shape = (150,150, 3), 
                                include_top = False, 
                                weights = None)
# pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [23]:
x = layers.Flatten()(last_output)
x = layers.Dropout(0.5)(x) 
x = layers.Dense(1024, activation='relu')(x)                 
x = layers.Dense (1, activation='sigmoid')(x)           
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [25]:
history = model.fit(
            train_gen,
            validation_data = val_gen,
            steps_per_epoch=20,
            epochs =10,
            validation_steps=5,
            verbose = 1)

Epoch 1/10
20/20 [==============================] - 64s 3s/step - loss: 0.6669 - accuracy: 0.9515 - val_loss: 0.3842 - val_accuracy: 0.9738
Epoch 2/10
20/20 [==============================] - 55s 3s/step - loss: 0.6254 - accuracy: 0.9755 - val_loss: 0.2463 - val_accuracy: 0.9762
Epoch 3/10
20/20 [==============================] - 56s 3s/step - loss: 0.5895 - accuracy: 0.9735 - val_loss: 0.1691 - val_accuracy: 0.9738
Epoch 4/10
20/20 [==============================] - 53s 3s/step - loss: 0.5520 - accuracy: 0.9754 - val_loss: 0.1358 - val_accuracy: 0.9744
Epoch 5/10
20/20 [==============================] - 52s 3s/step - loss: 0.5131 - accuracy: 0.9743 - val_loss: 0.1392 - val_accuracy: 0.9709
Epoch 6/10
20/20 [==============================] - 48s 2s/step - loss: 0.4775 - accuracy: 0.9733 - val_loss: 0.1313 - val_accuracy: 0.9738
Epoch 7/10
20/20 [==============================] - 50s 2s/step - loss: 0.4376 - accuracy: 0.9754 - val_loss: 0.1630 - val_accuracy: 0.9733
Epoch 8/10
20/20 [==

In [26]:
model.evaluate(test_gen)

20/20 [==============================] - 103s 5s/step - loss: 0.2026 - accuracy: 0.9757


[0.20261450111865997, 0.9757267236709595]